In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from skimage import io
import os
from PIL import Image
import random
import cv2

You should create a directoy with MSID name and create 4 folders as follows: normal, monkeypox, chickenpox, measles plus 4 more folders as follows: normal_augmented, monkeypox_augmented, chickenpox_augmented, measles_augmented so we have 8 folders in MSID you can change all this in below blocks.

Augmented size would be 48 but every 8 augmented image would be with one of the noise function 

In [ ]:
augmentation_batch_sizes = [293, 279, 107, 91]
SIZE = 224
AUGMENTED_SIZE = (48 // 6)
base_directory = '/content/drive/MyDrive/MSID/'
paths = ['normal', 'monkeypox', 'chickenpox', 'measles']
number_of_classes = len(paths)

Our six noise function would be: add_salt_and_pepper_noise, add_salt_noise, add_pepper_noise, add_gaussian_noise and change color

In [ ]:
def add_salt_and_pepper_noise(image):

    image = np.array(image)

    change_pixel_color(image, 255)

    change_pixel_color(image, 0)

    return image

def add_salt_noise(image):

    image = np.array(image)

    change_pixel_color(image, 255)

    return image

def add_pepper_noise(image):

    image = np.array(image)

    change_pixel_color(image, 0)

    return image

# change the color of a pixel which choose ranomly
def change_pixel_color(image, color=0):

    # Getting the dimensions of the image
    row , column, channels = image.shape

    # number of pixels which will be have noise
    number_of_pixels = np.random.randint(100, 350)

    for i in range(number_of_pixels):

        # Pick a random y coordinate
        y_coordinate = np.random.randint(0, row - 1)

        # Pick a random x coordinate
        x_coordinate = np.random.randint(0, column - 1)

        # Color that pixel to white
        image[y_coordinate][x_coordinate] = color

In [ ]:
def add_gaussian_noise(image):

    # get the shape of image
    row, column, channel = image.shape

    # initialize required variables to create guassian noise
    mean = 0
    variance = random.uniform(0.005, 0.02)
    sigma = variance ** 0.5

    # prepare the noise
    gaussian_noise = np.random.normal(mean,sigma,(row, column, channel))
    gaussian_noise = gaussian_noise.reshape(row, column, channel)

    # add the noise to the image
    noisy_image = image + gaussian_noise

    return noisy_image

In [ ]:
def change_color(image):
    image = np.array(image)
    # convert to grastyle
    bgr_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # add new dimontion to have equal shape like RGB
    return bgr_image

You can set your configuration for augmentation.

In [ ]:
def create_image_data_generators() -> list:
    # save all functions that we want apply on images
    preprocessing_functions = [add_gaussian_noise, add_salt_and_pepper_noise, add_salt_noise, add_pepper_noise, change_color, None]
    # create generators according to number of our functions
    generators = []
    for function in preprocessing_functions:

        generator = ImageDataGenerator(
            rescale=1.0/255.0,
            brightness_range=(0.5, 1),
            vertical_flip=True,
            horizontal_flip=True,
            channel_shift_range= random.uniform(0.0, 0.4),
            zoom_range=random.uniform(0.0, 0.4),
            rotation_range=90,
            height_shift_range=random.uniform(0.0, 0.4),
            width_shift_range=random.uniform(0.0, 0.4),
            shear_range=random.uniform(0.0, 0.4),
            preprocessing_function=function,
            fill_mode='reflect'
        )

        generators.append(generator)

    return generators

In [ ]:
def augmentation_images(path: str, augmentation_batch_size: int, generators: list) -> None:
    counter = 0
    for generator in generators:
        original_images = generator.flow_from_directory(directory=path,
                                                 batch_size=augmentation_batch_size,
                                                 target_size=(SIZE, SIZE),
                                                 color_mode="rgb",
                                                 save_to_dir=f'{path}_augmented',
                                                 save_prefix='aug',
                                                 save_format='png')
        for _ in original_images:

            counter += 1
            print(f'{original_images.save_to_dir}: {counter}')
            if counter > AUGMENTED_SIZE:
                break

        counter = 0

In [ ]:
generators = create_image_data_generators()
for i in range(number_of_classes):
    augmentation_images(base_directory + paths[i], augmentation_batch_sizes[i], generators)